# Benchmark data processing

Contains the code required to plot the figures in the docs.

* Build the benchmarks with CMake, using the `BOOST_MYSQL_BENCH` CMake option and the `boost_mysql_bench` CMake target.
* Run the benchmarks with `tools/ci/run_benchmarks.py`, which leaves a .txt file with the results in the current directory.
* Run this notebook to generate the graphs.


In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv('../private/benchmark-results.txt', header=None) # Update file path as required
df = df.set_index(df.index.map(lambda x: x % df.groupby(0).size()[0]))
df = df.pivot(columns=[0], values=[1])
df.columns = df.columns.get_level_values(0)
df

In [ ]:
def plot_result(df: pd.DataFrame, bench: str, title: str, ax):
    cols = [f'{bench}_boost', f'{bench}_libmysqlclient', f'{bench}_libmariadb']
    df = df[cols].rename(columns={
        f'{bench}_boost': 'Boost.MySQL',
        f'{bench}_libmysqlclient': 'libmysqlclient',
        f'{bench}_libmariadb': 'libmariadb'
    })
    mean_val = round(df.mean().mean())
    df.plot.box(ylim=(mean_val-150, mean_val+150), ax=ax, title=title, ylabel='Time (ms)')


fig, _ = plt.subplots(2, 2, figsize=(15, 15))

plot_result(df, bench='one_small_row', title='Reading one small row', ax=fig.axes[0])
plot_result(df, bench='one_big_row', title='Reading one big row', ax=fig.axes[1])
plot_result(df, bench='many_rows', title='Reading 5k rows', ax=fig.axes[2])
plot_result(df, bench='stmt_params', title='Statement with params', ax=fig.axes[3])
